In [177]:
import datetime as dt
from typing import List, Optional
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from abc import ABC, abstractmethod
import selenium
from selenium import webdriver

In [178]:
class Analisys:
    
    def __init__(self, source: str, chapter: str|None, analisys_name: str, analisys_cost:str, analisys_comment: str,):              

        self.source    = source
        self.chapter     = chapter
        self.analisys_name   = analisys_name
        self.analisys_cost   = analisys_cost
        self.analisys_comment  = analisys_comment

    def __repr__(self) -> str:
        return f"source = {self.source}, chapter = {self.chapter}, analisys_name = {self.analisys_name}, analisys_cost = {self.analisys_cost}, analisys_comment = {self.analisys_comment}"
    
    def to_dict(self) -> dict:
        return self.__dict__


#     def to_DB(self, ...):
#         ...

#     @classmethod
#     def from_DB(cls, ...):
#         ...


In [179]:
class WebDriver(ABC):

    def __init__(self):
        self.urls: List[str] = []
            
    def _scan_page(self, url:str):
        # Прописываем нестандартный заголовок, чтобы сайт не принял нас за бота, который парсит его данные :)
        headers = requests.utils.default_headers()
        headers.update({
                    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
                    "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36',
            })

        response = requests.get(url=url, headers=headers)

        if not response.ok:
            raise ValueError("no content to parse")

        return BeautifulSoup(response.content.decode(), "lxml")            

    @abstractmethod
    def get_analisys(self) :  
        ...


In [181]:


class DriverDiaglab(WebDriver):
    
    URL = 'https://diaglab.by/biochemical-test/'
    NAME = "https://diaglab.by"

    def __init__(self):
       ...
    
    def get_analisys(self) :
        pass
        
        


    def get_analisys1(self, urls):
        analisys_dict = {}
        i=0
        for url in urls:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')

            page_number = 1
            while True:
                product_cards = soup.find('div', class_='products-loop').find_all('div', class_='product-card')
                for product_card in product_cards:
                    artikul = product_card.find('div', class_='product-card__code').text
                    name = product_card.find(class_='product-card__excerpt').find('p').text
                    price = product_card.find(class_='product-card__price').text.replace('\xa0', '')
                    link_description = product_card.find('h3', class_='product-card__title').find('a').get('href')
                    delivery = product_card.find(class_='product-card__delivery').text
                    # print(url,artikul, name, price, link_description, delivery)
                    i+=1

                    # source = 'https://diaglab.by'
                    chapter = url
                    analisys_name = name
                    analisys_cost = price
                    analisys_comment = artikul,link_description,delivery
                    analisys = Analisys(self.NAME, chapter, analisys_name, analisys_cost, analisys_comment)
                    analisys_dict[i] = (analisys.to_dict())



                next_page = soup.find('a', class_='next page-numbers')
                if next_page:
                    page_number += 1
                    next_url = url + f'page/{page_number}/'
                    response = requests.get(next_url)
                    soup = BeautifulSoup(response.text, 'html.parser')
                else:
                    break
        
        return analisys_dict                

list_links_razdely = [
    'https://diaglab.by/biochemical-test/',
    'https://diaglab.by/immunology/',
    'https://diaglab.by/vitaminy/',
    'https://diaglab.by/gormonal-test/',
    'https://diaglab.by/gematology/',
    'https://diaglab.by/autoimmune-diseases/',
    'https://diaglab.by/blood-groups/',
    'https://diaglab.by/urine-examination/',
    'https://diaglab.by/gemostaz/',
    'https://diaglab.by/microscopy/',
    'https://diaglab.by/onkological-panel/',
    'https://diaglab.by/allergology/',
    'https://diaglab.by/infectious-diseases/',
    'https://diaglab.by/analiz-kala/',
    'https://diaglab.by/mikrobiologicheskie-issledovaniya/',
    'https://diaglab.by/lekarstvennyj-monitoring/',
    'https://diaglab.by/mikroelementy/',
    'https://diaglab.by/pcr-diagnostika/',
    'https://diaglab.by/kompleksy-analizov/'
]

# d = DriverDiaglab()
# d.get_analisys()

e = DriverDiaglab()
df_diaglab = pd.DataFrame.from_dict(e.get_analisys1(list_links_razdely).values())
pd.set_option('display.max_colwidth', None)
df_diaglab.head(20)


,source,chapter,analisys_name,analisys_cost,analisys_comment
0,https://diaglab.by,https://diaglab.by/biochemical-test/,Определение железа в сыворотке крови,2.12 руб.,"(Артикул: 1033, https://diaglab.by/iron/, 1 рабочий день (не включая день взятия биоматериала) )"
1,https://diaglab.by,https://diaglab.by/biochemical-test/,"Определение электролитов в сыворотке крови: Калий (К+, Potassium), Натрий (Na+, Sodium), Хлор (Сl-, Chloride)",3.98 руб.,"(Артикул: 1027, https://diaglab.by/elektrolity/, 1 рабочий день (не включая день взятия биоматериала) )"
2,https://diaglab.by,https://diaglab.by/biochemical-test/,Определение магния в сыворотке крови,1.68 руб.,"(Артикул: 1031, https://diaglab.by/magnesium/, 1 рабочий день (не включая день взятия биоматериала) )"
3,https://diaglab.by,https://diaglab.by/biochemical-test/,Определение глюкозы в сыворотке крови,1.54 руб.,"(Артикул: 1008, https://diaglab.by/glucosae/, 1 рабочий день (не включая день взятия биоматериала) )"
4,https://diaglab.by,https://diaglab.by/biochemical-test/,Определение Кальция общего в сыворотке крови,1.55 руб.,"(Артикул: 1028, https://diaglab.by/calcium-total/, 1 рабочий день (не включая день взятия биоматериала) )"
5,https://diaglab.by,https://diaglab.by/biochemical-test/,Определение холестерина общего в сыворотке крови,2.36 руб.,"(Артикул: 1021, https://diaglab.by/cholesterol/, 1 рабочий день (не включая день взятия биоматериала) )"
6,https://diaglab.by,https://diaglab.by/biochemical-test/,Определение фосфора неорганического в сыворотке крови,1.96 руб.,"(Артикул: 1032, https://diaglab.by/phosphate-inorganic/, 1 рабочий день (не включая день взятия биоматериала) )"
7,https://diaglab.by,https://diaglab.by/biochemical-test/,Определение аланинаминотрансферазы в сыворотке крови,1.87 руб.,"(Артикул: 1010, https://diaglab.by/alt/, 1 рабочий день (не включая день взятия биоматериала) )"
8,https://diaglab.by,https://diaglab.by/biochemical-test/,Определение аспартатаминотрансферазы в сыворотке крови,1.73 руб.,"(Артикул: 1009, https://diaglab.by/ast/, 1 рабочий день (не включая день взятия биоматериала) )"
9,https://diaglab.by,https://diaglab.by/biochemical-test/,Определение общего билирубина в сыворотке крови,1.95 руб.,"(Артикул: 1003, https://diaglab.by/bilirubin-total/, 1 рабочий день (не включая день взятия биоматериала) )"


In [176]:
df_diaglab.iloc[390:450]

,source,chapter,analisys_name,analisys_cost,analisys_comment
390,https://diaglab.by,https://diaglab.by/kompleksy-analizov/,"ФСГ, ЛГ, эстрадиол, ТТГ, пролактин, тестостерон общий.",84.56 руб.,"(Артикул: 2021, https://diaglab.by/zhenskie-gormony-skrining-2/, 1 рабочий день (не включая день взятия биоматериала) )"
391,https://diaglab.by,https://diaglab.by/kompleksy-analizov/,"Витамин Д, ОАК без дифференцировки лейкоцитарной формулы, ОАМ, белок общий, общий билирубин, мочевина, общий холестерин, триглицериды, ЛПВП, ЛПНП, глюкоза, АЛТ, АСТ, железо, кальций общий, магний,...",82.01 руб.,"(Артикул: 2041, https://diaglab.by/dietolog/, 1 рабочий день (не включая день взятия биоматериала) )"
392,https://diaglab.by,https://diaglab.by/kompleksy-analizov/,"Белок общий, альбумин, общий билирубин, прямой билирубин, мочевина, креатинин, мочевая кислота, глюкоза, АСТ, АЛТ, амилаза общая, щелочная фосфатаза, ГГТ, ЛДГ, КФК, общий холестерин, триглицериды, ЛПВП,...",178.44 руб.,"(Артикул: 2019, https://diaglab.by/chekap-zhenshhiny/, 3 рабочих дня (не включая день взятия биоматериала) )"
393,https://diaglab.by,https://diaglab.by/kompleksy-analizov/,"Общий анализ крови с лейкоцитарной формулой, общий анализ мочи, общий билирубин, прямой билирубин, глюкоза, креатинин, мочевина, АСТ, АЛТ, щелочная фосфатаза",25.53 руб.,"(Артикул: 2031, https://diaglab.by/deti-kompleks/, 1 рабочий день (не включая день взятия биоматериала) )"
394,https://diaglab.by,https://diaglab.by/kompleksy-analizov/,"Общий холестерин, триглицериды, ЛПВП, ЛПНП",10.88 руб.,"(Артикул: 2014, https://diaglab.by/lipidnyj-profil/, 1 рабочий день (не включая день взятия биоматериала) )"
395,https://diaglab.by,https://diaglab.by/kompleksy-analizov/,"Кортизол, ДГЭА-С, СРБ, ТТГ",38.48 руб.,"(Артикул: 2030, https://diaglab.by/stress-kompleks/, 1 рабочий день (не включая день взятия биоматериала) )"
396,https://diaglab.by,https://diaglab.by/kompleksy-analizov/,"Белок общий, альбумин, общий билирубин, прямой билирубин, мочевина, креатинин, мочевая кислота, глюкоза, АСТ, АЛТ, амилаза общая, щелочная фосфатаза, ГГТ, ЛДГ, КФК, общий холестерин, триглицериды, ЛПВП,...",178.84 руб.,"(Артикул: 2020, https://diaglab.by/chekap-muzhchiny/, 3 рабочих дня (не включая день взятия биоматериала) )"
397,https://diaglab.by,https://diaglab.by/kompleksy-analizov/,"17-ОН прогестерон, ФСГ, ЛГ, пролактин, тестостерон общий, эстрадиол, ДГЭА-С, ТТГ, Т4 св., кортизол",130.84 руб.,"(Артикул: 2033, https://diaglab.by/sostoyanie-gormonalnogo-fona/, 5 рабочих дней (не включая день взятия биоматериала) )"
398,https://diaglab.by,https://diaglab.by/kompleksy-analizov/,"Общий билирубин, прямой билирубин, АЛТ, АСТ, ГГТП, щелочная фосфатаза, общий холестерин, белок общий",15.40 руб.,"(Артикул: 2029, https://diaglab.by/liver-kompleks/, 1 рабочий день (не включая день взятия биоматериала) )"
399,https://diaglab.by,https://diaglab.by/kompleksy-analizov/,"Жидкостная цитология, вирус папилломы человека — 17 типов.",69.99 руб. 59.49 руб.,"(Артикул: 2066, https://diaglab.by/screening-cancer/, 3 рабочих дня (не включая день взятия биоматериала) )"
